In [1]:
import numpy as np
import pandas as pd

path_to_csv = "../data/mortgage_transformed.csv"
df = pd.read_csv(path_to_csv)
df.set_index(df.pop("Unnamed: 0"), inplace=True)

X = df.dropna(axis=0)
y = X.pop("result")

# Building a better model

We will try to select variables that may be related to the output as well as feature enginnering new columns. 

Looking at the available columns, we can create variables such as time elapsed since loan was taken out, change in FICO, LTV or HPI.

In [4]:
X["time_since_start"] = X.time - X.first_time
X["balance_change"] = X.balance_orig_time-X.balance_time
X["LTV_change"] = X.LTV_time - X.LTV_orig_time

In [5]:
from sklearn.model_selection import train_test_split

better_columns = ["time_since_start", "balance_change", "LTV_change"]

X_train, X_valid, y_train, y_valid = train_test_split(
    X[better_columns], y,
    train_size=0.7,
    test_size=0.3)

In [ ]:
# we will try to select a better model using accuracy scoring
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

def score_model(X_train, y_train, X_valid, y_valid, n_estimators=100, learning_rate=0.1):
    model = XGBClassifier(
                n_estimators = n_estimators,
                learning_rate = learning_rate,
                n_jobs = 5,
                eval_metric='mlogloss'
            )
    
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    
    accuracy = accuracy_score(predictions, y_valid)
    return accuracy

In [15]:
# we will try different number of estimators to see, which is the best fit
estimator_count = [25, 50, 100, 200, 300, 400, 800]

last_accuracy = 0
for n_estimators in estimator_count:
    accuracy = score_model(
        X_train, y_train, X_valid, y_valid,
        n_estimators=n_estimators,
        learning_rate=0.3 # let's speed things up
    )
    if accuracy > last_accuracy:
        last_accuracy = accuracy
    else:
        print(f"The ideal number of estimators is {n_estimators} with an accuracy of {accuracy}")
        break

C:\Users\vojte\Documents\GitHub\mortgage-default-prediction\venv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\vojte\Documents\GitHub\mortgage-default-prediction\venv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[21:52:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\vojte\Documents\GitHub\mortgage-default-prediction\venv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\vojte\Documents\GitHub\mortgage-default-prediction\venv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[21:52:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
The ideal number of estimators is 50 with an accuracy of 0.6520445277196076


In [21]:
model = XGBClassifier(
                n_estimators = 50,
                learning_rate = 0.1,
                n_jobs = 5,
                eval_metric='mlogloss'
        )

model.fit(X_train, y_train)
predictions = model.predict(X_valid)

print(f"The accuracy score is {accuracy_score(predictions, y_valid)}")

path_to_model = "../models/second_model.json"
model.save_model(path_to_model)

The accuracy score is 0.6546346302215365


# Summary

This model makes correct predictions roughly 65% of the time.

In total terms, it is roughly 48% at predicting whether a debt will be paid off, or a default will happen. This is much better than the previous one, though more work can be done to make it better.

We shall try to have a deeper look at the relationships between the variables in the next model.